In [1]:
%reset -f

In [2]:
import numpy                 as np
import pandas                as pd
import statsmodels.api       as sm

In [3]:
dataset = pd.read_excel('cps09mar.xlsx')

In [4]:
dataset.columns

Index(['age', 'female', 'hisp', 'education', 'earnings', 'hours', 'week',
       'union', 'uncov', 'region', 'race', 'marital'],
      dtype='object')

In [5]:
#Exercise 3.24
#[1]
#generate variables
dataset['lwage'] = np.log(dataset.earnings)
dataset['experience'] = dataset.age - dataset.education - 6
dataset['experience2'] = np.square(dataset.experience)/100

In [6]:
dataset.head()

,age,female,hisp,education,earnings,hours,week,union,uncov,region,race,marital,lwage,experience,experience2
0,52,0,0,12,146000,45,52,0,0,1,1,1,11.891362,34,11.56
1,38,0,0,18,50000,45,52,0,0,1,1,1,10.819778,14,1.96
2,38,0,0,14,32000,40,51,0,0,1,1,1,10.373491,18,3.24
3,41,1,0,13,47000,40,52,0,0,1,1,1,10.757903,22,4.84
4,42,0,0,13,161525,50,52,1,0,1,1,1,11.992415,23,5.29


In [7]:
#Restrict the to individuals with less than 45 years experience
dataset = dataset[dataset['experience']<45]

Estimate the regression
<font size = 4>$$ \log(wage)=\beta_0+\beta_1education+\beta_2experience+\beta_3experience^2+e $$</font>

In [8]:
X = dataset[['education', 'experience', 'experience2']]
X = sm.add_constant(X)
Y = dataset[['lwage']]

In [9]:
OLS_model = sm.OLS(Y, X)
OLS_1 = OLS_model.fit()
print(OLS_1.summary())
print("SSR is:",OLS_1.ssr)

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     5814.
Date:                Sun, 08 Oct 2023   Prob (F-statistic):               0.00
Time:                        18:48:52   Log-Likelihood:                -45028.
No. Observations:               49477   AIC:                         9.006e+04
Df Residuals:                   49473   BIC:                         9.010e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.4178      0.018    479.980      

<font size=4>So now we know that $$R^2=0.261,\,and\,SSR=17882.08$$</font>


In [10]:
#[2]
#Regress log(wage) on experience and experience^2
X2 = dataset[['experience', 'experience2']]
X2 = sm.add_constant(X2)

In [11]:
OLS_model2 = sm.OLS(Y, X2)
OLS_2 = OLS_model2.fit()
print(OLS_2.summary())
res_2 = OLS_2.resid #Save the residual from the regression

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     946.4
Date:                Sun, 08 Oct 2023   Prob (F-statistic):               0.00
Time:                        18:48:52   Log-Likelihood:                -51570.
No. Observations:               49477   AIC:                         1.031e+05
Df Residuals:                   49474   BIC:                         1.032e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          10.2075      0.011    924.044      

In [12]:
#Regress education on experience and experience^2
Y3 = dataset[['education']]

In [13]:
OLS_model3 = sm.OLS(Y3, X2)
OLS_3 = OLS_model3.fit()
print(OLS_3.summary())
res_3 = OLS_3.resid #save the residual from the regression

                            OLS Regression Results                            
Dep. Variable:              education   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     591.3
Date:                Sun, 08 Oct 2023   Prob (F-statistic):          1.71e-254
Time:                        18:48:52   Log-Likelihood:            -1.1860e+05
No. Observations:               49477   AIC:                         2.372e+05
Df Residuals:                   49474   BIC:                         2.372e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          14.3906      0.043    336.111      

In [14]:
#Regress residules on residules
OLS_model4 = sm.OLS(res_2, sm.add_constant(res_3))
OLS_4 = OLS_model4.fit()
print(OLS_4.summary())
print("SSR is:",OLS_4.ssr)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                 1.498e+04
Date:                Sun, 08 Oct 2023   Prob (F-statistic):               0.00
Time:                        18:48:52   Log-Likelihood:                -45028.
No. Observations:               49477   AIC:                         9.006e+04
Df Residuals:                   49475   BIC:                         9.008e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.876e-15      0.003   2.17e-12      1.0

<font size=4>From the result, we know that$$the\,estimated\,coefficient=0.1244,\,R^2=0.232,\,and\,SSR=17882.08$$</font>
<font size=4>Thus, the estimate is the same as [1], the SSR is also the same, and R^2 is less than [1]</font>

[3]  
<font size=4>The reason why the estimate in [2] and [1] are the same:   
Let set education as matrix $X_1$, [experience,experience2] as matrix $X_2$.   
In [1], we regress $Y = X_1\beta_1+X_2\beta_2+e$<br>
$$
(\hat\beta_1,\hat\beta_2)=\underset{\beta_1,\beta_2}{{\arg\min}\,}SSE(\beta_1,\beta_2)\\  
\hat\beta_1=\underset{\beta_1}{{\arg\min}\,}(\underset{\beta_2}{\min}\,SSE(\beta_1,\beta_2))\\=(X_1'M_2X_1)^{-1}{X_1'M_2Y}
$$    
In [2], firstly we run a regression $Y=X_2\theta+u$<br>
we can get:
$$
\hat\theta=(X_2'X_2)^{-1}X_2'Y,\\
\hat u=Y-X_2(X_2'X_2)^{-1}X_2'Y=M_2Y
$$
Secondly we run a regression $X_1=X_2\lambda+\epsilon$<br>
we can get:
$$
\hat\lambda=(X_2'X_2)^{-1}X_2'X_1,\\
\hat\epsilon=Y-X_2(X_2'X_2)^{-1}X_2'X_1=M_2 X_1
$$  
Lastly we run a regression $\hat u=\hat\epsilon\alpha+\omega$<br>
Let's calculate the estimate $\hat\alpha$
$$
\hat\alpha=(\hat\epsilon'\hat\epsilon)^{-1}\hat\epsilon'\hat u\\=((M_2X_1)'M_2X1)^{-1}{(M_2X_1)'M_2Y}\\=(X_1'M_2X_1)^{-1}{X_1'M_2Y}\\
so\,we\,find\,\hat\alpha=\hat\beta_1
$$   
</font>

In [15]:
#Exercise 3.25
#Let's do it in matrix form
Y = dataset['lwage']
X1 = dataset['education']
dataset['education2'] = np.square(dataset.experience)/100
X12 = dataset['education2']
X2 = dataset['experience']
X22 = dataset['experience2']
X = np.column_stack((np.ones(Y.shape[0]), X1, X2,X22))

In [16]:
#Now, we can caculate beta using formula (X'X)^(-1)X'Y
beta = np.linalg.inv(X.T @ X) @ X.T @ Y
print(beta)

[ 8.41775154  0.12436935  0.04487859 -0.077773  ]


In [17]:
#Set Y_hat
Y_hat = X @ beta

In [18]:
#Set Residule_hat
e_hat = Y - Y_hat

In [19]:
#Now we have every matrix we need, we can calculate the questions [a]-[g]:
q_a = np.sum(e_hat)
q_b = X1.T @ e_hat
q_c = X2.T @ e_hat
q_d = X12.T @ e_hat
q_e = X22.T @ e_hat
q_f = Y_hat.T @ e_hat
q_g = e_hat.T @ e_hat
print("Answer of [a] is:",q_a)
print("Answer of [b] is:",q_b)
print("Answer of [c] is:",q_c)
print("Answer of [d] is:",q_d)
print("Answer of [e] is:",q_e)
print("Answer of [f] is:",q_f)
print("Answer of [g] is:",q_g)

Answer of [a] is: 2.589040093425865e-11
Answer of [b] is: 7.3687260737642646e-09
Answer of [c] is: 3.4415279515087605e-09
Answer of [d] is: -4.774847184307873e-10
Answer of [e] is: -4.774847184307873e-10
Answer of [f] is: 1.3278622645884752e-09
Answer of [g] is: 17882.080177176773


<font size=4>
The results are consistent with the theoretical properties of OLS,
$$
X'\hat e=0, \\
\hat Y'\hat e=(X\hat\beta)'\hat e=\hat\beta' X'\hat e=0
$$
So we can see answers for [a] - [f] are very close to 0,<br>
[g] is SSR, which is the same as Exercise 3.24

In [20]:
#Exerceise 3.26
#[a]. restrict the data only for white male Hispanics, and create dummy variables
dataset = dataset.loc[(dataset['female'] == 0) & (dataset['race'] == 1) & (dataset['hisp'] == 1)]
region_dummies = pd.get_dummies(dataset['region'], prefix='region')
marital_dummies = pd.get_dummies(dataset['marital'], prefix='marital')
dataset = pd.concat([dataset, region_dummies, marital_dummies], axis=1)

In [21]:
dataset.head()

,age,female,hisp,education,earnings,hours,week,union,uncov,region,...,region_2,region_3,region_4,marital_1,marital_2,marital_3,marital_4,marital_5,marital_6,marital_7
112,53,0,1,9,50000,70,52,0,0,1,...,0,0,0,1,0,0,0,0,0,0
267,21,0,1,13,30000,40,52,0,0,1,...,0,0,0,0,0,0,0,0,0,1
271,40,0,1,20,75000,40,52,0,0,1,...,0,0,0,1,0,0,0,0,0,0
460,27,0,1,12,18500,40,52,0,0,1,...,0,0,0,0,0,0,0,0,0,1
461,55,0,1,12,22000,40,49,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [22]:
X = dataset[['region_1','region_3','region_4','marital_1','marital_2','marital_3','marital_4','marital_5','marital_6','education','experience', 'experience2']]
X = sm.add_constant(X)
Y = dataset[['lwage']]
OLS_model5 = sm.OLS(Y, X)
OLS_5 = OLS_model5.fit()
print(OLS_5.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.271
Model:                            OLS   Adj. R-squared:                  0.269
Method:                 Least Squares   F-statistic:                     125.7
Date:                Sun, 08 Oct 2023   Prob (F-statistic):          1.33e-267
Time:                        18:48:52   Log-Likelihood:                -3545.1
No. Observations:                4065   AIC:                             7116.
Df Residuals:                    4052   BIC:                             7198.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.7284      0.056    157.003      

<font size=4>
    In Exercise 3.26 [b], I tried STATA, and got the same result. Please check the do-file.
</font>